In [ ]:
import h5py
import condor
import copy
import numpy as np
import condor.utils.linalg as linalg
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm

In [ ]:
fullp_res_edge   = lambda det,w:  1./linalg.length(det.get_q_max(w, pos='edge')/(2*np.pi))
fullp_res_corner = lambda det,w:  1./linalg.length(det.get_q_max(w, pos='corner')/(2*np.pi))

In [ ]:
def plot_diffraction(data, mask=None):
    if mask is None:
        mask = np.ones_like(data).astype(np.bool)
    image = np.ma.array(np.copy(data).astype(np.float), mask=~mask)
    image.data[image.data<0.5] = 1e-5
    palette = copy.copy(cm.magma)
    palette.set_bad('w', 1.)
    palette.set_under('0.9',1.)
    plt.figure()
    plt.imshow(image, norm=colors.LogNorm(vmin=1), interpolation='none', cmap=palette)
    plt.axis('off')
    cb = plt.colorbar(pad=0)
    cb.ax.set_ylabel('Intensity [photons / px]')
    cb.outline.set_visible(False)
    plt.tight_layout()
    plt.show()

# Parameters

In [ ]:
# Detector (pnCCD)
pixelsize = 75e-6
nx,ny = (1024,1024)
detector_distance = 300e-3

In [ ]:
# Source
photon_energy = 3500 # [eV]
ph = condor.utils.photon.Photon(energy_eV=photon_energy)
wavelength = ph.get_wavelength()
fluence = 1e15 #[ph/um2]
focus_diameter = 0.2e-6
pulse_energy = fluence * ph.get_energy() * (np.pi*((1e6*focus_diameter/2.)**2)) # [J]

In [ ]:
# Sample
pdb_id = '1FFK'
sample_size = 18e-9

# Run simulation

In [ ]:
src = condor.Source(wavelength=wavelength, pulse_energy=pulse_energy, focus_diameter=focus_diameter)
det = condor.Detector(distance=detector_distance, pixel_size=pixelsize, nx=nx, ny=ny, noise="poisson")
par = condor.ParticleAtoms(pdb_id=pdb_id,rotation_formalism="random")
E = condor.Experiment(source=src, particles={"particle_atoms":par}, detector=det)
o = E.propagate()

# Output

In [ ]:
print "Photon energy: %d eV" %photon_energy
print "Fluence: %g ph/um2" %fluence
print "Pulse energy: %.4f mJ" %(1e3*pulse_energy)
print "Sample size: %d nm" %(1e9*sample_size)
print "Detector distance: %d mm" %(1e3*detector_distance)
print "Full period resolution (corner): %.2f nm" %(1e9*fullp_res_corner(det,wavelength))
print "Full period resolution (edge): %.2f nm" %(1e9*fullp_res_edge(det,wavelength))
print "Nr. of resolution elements: %.2f" %(sample_size / fullp_res_corner(det,wavelength)*2)
plot_diffraction(o['entry_1']['data_1']['data'])
print "\n"

# Write to CXI file

In [ ]:
W = condor.utils.cxiwriter.CXIWriter("../data/single_diffraction_pattern.h5")
W.write(o)
W.close()